In [9]:
import numpy as np
import pandas as pd
from collections import defaultdict
import tensorflow as tf

import time 
import os

# !ls
os.chdir('/content/drive/')

start_time = time.time()

FEATURE_PATH = 'nn/smart_city/feature_scene1.csv'
CHECKPOINT_PATH = 'nn/smart_city/models/scene1'
MODEL_PATH = os.path.join(CHECKPOINT_PATH, 'scene1_model.meta')
RESULT_PATH = 'nn/smart_city/result_scene1.csv'

if __name__ == '__main__':
  features = pd.read_csv(FEATURE_PATH)
  coordinates_dict = defaultdict(list)
  
  with tf.Session() as sess:
    saver = tf.train.import_meta_graph(MODEL_PATH)
    saver.restore(sess, tf.train.latest_checkpoint(CHECKPOINT_PATH))
    
    W1 = sess.run('W1:0')
    b1 = sess.run('b1:0')
    W2 = sess.run('W2:0')
    b2 = sess.run('b2:0')
    
    # 求各个文件夹下文件的各个特征给出坐标的总和
    for row in features.values:
      feature = [row[1:].tolist()]
#       print(feature, type(feature))
      
      X = tf.Variable(feature)
      temp = tf.matmul(X, W1) + b1
      _y = tf.matmul(temp, W2) + b2

      sess.run(tf.global_variables_initializer())
      _y = _y.eval()[0]
      print(_y)
      
      coordinates_dict[row[0]].append(_y)
    
    # 求各个特征给出的坐标均值作为文件夹对应相机的最终坐标
    coordinate_dict = {}
    for key, value in coordinates_dict:
      sum = np.array([0, 0, 0])
      for v in value:
        sum += v
      
      coordinate_dict[key] = sum / values.shape[0]
      
    dir_name = list(coordinate.keys())
    coordinate = list(coordinate_dict.values())
    
    rows = []
    for i, v in enumerate(coordinate):
      rows.append(dir_name + v)
    
    with open(RESULT_PATH, 'a+') as f:
      writer = csv.writer(f)
      writer.writerows(rows)

#     mse = ((labels - _y) ** 2).sum(axis=0).sum(axis=0) // labels.shape[0]
#     print('mse =', mse)

    print('time = %ss'%(time.time() - start_time))
    
# 参考
# 1、Python的numpy库中将矩阵转换为列表等函数：https://blog.csdn.net/flying_sfeng/article/details/66478616

FailedPreconditionError: ignored